In [1]:
import os
import pandas as pd
import numpy as np
import mlflow
from mlflow.tracking import MlflowClient
from datetime import datetime
import re

In [2]:

# Adjust if your mlruns is elsewhere
MLRUNS_DIR_CANDIDATES = [
    # "/home/zano/Documents/TESI/FOLDER_CINECA/notebooks/mlruns",
    "/home/zano/Documents/TESI/FOLDER_CINECA/mlruns"
]

# def pick_existing_dir(candidates):
#     for d in candidates:
#         if os.path.isdir(d):
#             return d
#     raise FileNotFoundError(f"No mlruns directory found in: {candidates}")
mlruns_dir = MLRUNS_DIR_CANDIDATES[0] #pick_existing_dir(MLRUNS_DIR_CANDIDATES)
tracking_uri = f"file://{mlruns_dir}"
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient()

experiments = client.search_experiments() 

# --- Configuration for Filtering ---
EXCLUDE_PARAM_KEYS = {"seed", "random_state"}
EXCLUDE_PARAM_PREFIXES = ["debug_", "temp_"]
EXCLUDE_PARAM_REGEX = [r"^unused_.*"]

# --CONFIG FOR METRIC FILTERING
EXCLUDE_METRIC_KEYS = {"train_loss_step", "val_loss_step"}
EXCLUDE_METRIC_PREFIXES = ["raw_", "tmp_"]
EXCLUDE_METRIC_REGEX = [r"val_fold",r".*loss*"]

# --- Data Loading ---
rows = []
for exp in experiments:
    exp_id = exp.experiment_id
    runs = client.search_runs([exp_id], max_results=1000)
    for run in runs:
        info = run.info
        data = run.data
        params = {f"param.{k}": v for k, v in data.params.items()}
        metrics = {f"metric.{k}": v for k, v in data.metrics.items()}
        tags = {f"tag.{k}": v for k, v in data.tags.items()}
        row = {
            "experiment_id": exp_id,
            "experiment_name": exp.name,
            "run_id": info.run_id,
            "run_name": data.tags.get("mlflow.runName", ""),
            "artifact_uri": info.artifact_uri,
        }
        row.update(params)
        row.update(metrics)
        row.update(tags)
        rows.append(row)

# --- DataFrame Creation and Filtering ---
df = pd.DataFrame(rows).sort_values(["experiment_name"])
print(df.shape)
print(df.info())

display(df)

(217, 204)
<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 55 to 203
Columns: 204 entries, experiment_id to metric.test_auc_fold_0
dtypes: float64(153), object(51)
memory usage: 347.5+ KB
None


,experiment_id,experiment_name,run_id,run_name,artifact_uri,param.use_crop,param.test_pat_ids_per_fold,param.crop_percentage,param.dropout_rate,param.creation_time,...,metric.test_recall_fold_5,metric.test_mcc_fold_5,metric.test_mcc_fold_2,metric.test_precision_fold_5,metric.test_precision_fold_2,metric.test_auc_fold_7,metric.test_auc_fold_6,metric.test_recall_fold_7,metric.test_auc_fold_5,metric.test_auc_fold_0
55,788257751769076553,DS1_3c_MSA-P_vs_PD_supervised,6796411cc98f49e6af7dc871aaf85f3b,Densenet121_oversamp_torchvision_10-13_at:22-0...,file:///home/zano/Documents/TESI/FOLDER_CINECA...,False,NaN,0.95,0.2,22:09:14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,788257751769076553,DS1_3c_MSA-P_vs_PD_supervised,b33b0bb2146b4e9b82da134e99ba066d,Densenet169_mixup0.2_oversamp_torchvision_10-1...,file:///home/zano/Documents/TESI/FOLDER_CINECA...,False,NaN,0.95,0.2,00:00:35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,788257751769076553,DS1_3c_MSA-P_vs_PD_supervised,4c6d7ae304dd4b9889c0e3925a9ff53c,Densenet169_oversamp_torchvision_10-14_at:11-5...,file:///home/zano/Documents/TESI/FOLDER_CINECA...,False,NaN,0.95,0.2,11:55:02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,788257751769076553,DS1_3c_MSA-P_vs_PD_supervised,a57283a2c763498aa0b6d0d62a3514f9,Resnet18_oversamp_torchvision_10-14_at:12-25-18,file:///home/zano/Documents/TESI/FOLDER_CINECA...,False,NaN,0.95,0.2,12:25:18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,788257751769076553,DS1_3c_MSA-P_vs_PD_supervised,7fc11b854e3d42f69f9d7835c2fbfa18,ViT_monai_10-14_at:13-28-54,file:///home/zano/Documents/TESI/FOLDER_CINECA...,False,NaN,0.95,0.2,13:28:54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,650492548885392028,pretrained_subslice_MSA_vs_PD_supervised,f8bc44c3a3bb42c7a050e4fb5daa593d,Densenet121_oversamp_TL_pretrained:imagenet-mi...,file:///leonardo_work/pMI24_EleBr_1/lzanotto/F...,NaN,NaN,NaN,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,650492548885392028,pretrained_subslice_MSA_vs_PD_supervised,d1413c9a42af466e997d0de502529d5b,Densenet121_oversamp_TL_pretrained:microscopyn...,file:///leonardo_work/pMI24_EleBr_1/lzanotto/F...,NaN,NaN,NaN,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,650492548885392028,pretrained_subslice_MSA_vs_PD_supervised,49edb9eb8aed422fa6e02b8bf233ddb6,Densenet169_oversamp_TL_pretrained:torchvision...,file:///leonardo_work/pMI24_EleBr_1/lzanotto/F...,NaN,NaN,NaN,0.2,19:38:53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,650492548885392028,pretrained_subslice_MSA_vs_PD_supervised,41ce2810528a45408c81a87ca6940387,Resnet18_oversamp_TL_pretrained:microscopynet_...,file:///leonardo_work/pMI24_EleBr_1/lzanotto/F...,NaN,NaN,NaN,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import numpy as np # Make sure numpy is imported

# Replace empty strings '' with the standard np.nan
df = df.replace('', np.nan)

import math # We'll need this to round up

# 1. Get the total number of rows (runs)
total_rows = len(df)

# 2. Calculate the threshold: 5% of total rows.
# This is the minimum *non-missing* values a column must have to be kept.
# We use math.ceil() to round up. e.g., if 5% is 10.8, we need at least 11 values.
min_non_nan_count = math.ceil(total_rows * 0.10)

print(f"Total rows: {total_rows}")
print(f"Columns will be KEPT if they have at least {min_non_nan_count} non-NaN values.")

# 3. Apply the drop
# Keep track of columns before for comparison
cols_before_drop = set(df.columns)

# axis='columns' applies this to columns, not rows
# thresh=... tells pandas the minimum *non-NaN* count to keep the column
df_cleaned = df.dropna(axis='columns', thresh=min_non_nan_count)

# 4. Report what happened
cols_after_drop = set(df.columns)
dropped_cols = cols_before_drop - cols_after_drop

print(f"\nDropped {len(dropped_cols)} columns with >95% missing values.")
if dropped_cols:
    print(f"Columns dropped: {list(dropped_cols)}")

# See the new shape
print(f"\nNew DataFrame shape: {df.shape}")

Total rows: 217
Columns will be KEPT if they have at least 22 non-NaN values.

Dropped 0 columns with >95% missing values.

New DataFrame shape: (217, 204)


In [4]:
param_cols_to_drop = []
for col in df.columns:
    if col.startswith("param."):
        param_name = col.replace("param.", "", 1)
        if (param_name in EXCLUDE_PARAM_KEYS or
            any(param_name.startswith(p) for p in EXCLUDE_PARAM_PREFIXES) or
            any(re.search(rx, param_name) for rx in EXCLUDE_PARAM_REGEX)):
            param_cols_to_drop.append(col)

metric_cols_to_drop = []
for col in df.columns:
    if col.startswith("metric."):
        metric_name = col.replace("metric.", "", 1)
        if (metric_name in EXCLUDE_METRIC_KEYS or
            any(metric_name.startswith(p) for p in EXCLUDE_METRIC_PREFIXES) or
            any(re.search(rx, metric_name) for rx in EXCLUDE_METRIC_REGEX)):
            metric_cols_to_drop.append(col)

cols_to_drop = param_cols_to_drop + metric_cols_to_drop
df.drop(columns=cols_to_drop, inplace=True)

print(f"Dropped {len(cols_to_drop)} parameter/metric columns based on exclusion rules.")
print(df.shape)

#drop columns with loss in their name
loss_cols = [col for col in df.columns if 'loss' in col.lower()]
df.drop(columns=loss_cols, inplace=True, errors='ignore')
print(f"Dropped {len(loss_cols)} columns containing 'loss': {loss_cols}")
print(df.shape)

print(f"After filtering, {len(df)} runs remain with experiment names starting with 'SL' or 'pretrained'")


Dropped 82 parameter/metric columns based on exclusion rules.
(217, 122)
Dropped 0 columns containing 'loss': []
(217, 122)
After filtering, 217 runs remain with experiment names starting with 'SL' or 'pretrained'


In [9]:
# Create a boolean mask for rows to keep
# keep only rows with experiment name starting with SL, pretrained, or DS
mask = (df['experiment_name'].str.startswith('SL', na=False) |
        df['experiment_name'].str.startswith('pretrained', na=False) |
        df['experiment_name'].str.startswith('DS', na=False))

# Apply the filter
n_cols_before = len(df.columns)
df = df[mask]
n_cols_after = len(df.columns)
print(f"Dropped {n_cols_before - n_cols_after} columns")
print(df.shape)

Dropped 0 columns
(212, 123)


In [13]:
import re

# 1. Compile a regex pattern to find 'fold_' + one or more digits
pattern = re.compile(r'fold_\d+')
# 2. Build a list of columns where the pattern is found ANYWHERE
columns_to_drop = [col for col in df.columns if pattern.search(col)]
# 3. Drop the identified columns
df = df.drop(columns=columns_to_drop)

print(f"Dropping {len(columns_to_drop)} columns containing 'fold': {columns_to_drop}")
# 2. Drop the identified columns

Dropping 32 columns containing 'fold': ['metric.test_recall_fold_4', 'metric.test_precision_fold_0', 'metric.test_recall_fold_1', 'metric.test_mcc_fold_0', 'metric.test_recall_fold_6', 'metric.test_auc_fold_3', 'metric.test_recall_fold_0', 'metric.test_auc_fold_4', 'metric.test_precision_fold_3', 'metric.test_mcc_fold_7', 'metric.test_auc_fold_2', 'metric.test_mcc_fold_3', 'metric.test_precision_fold_4', 'metric.test_mcc_fold_4', 'metric.test_mcc_fold_6', 'metric.test_precision_fold_6', 'metric.test_precision_fold_7', 'metric.test_recall_fold_3', 'metric.test_auc_fold_1', 'metric.test_mcc_fold_1', 'metric.test_precision_fold_1', 'metric.test_recall_fold_2', 'metric.test_recall_fold_5', 'metric.test_mcc_fold_5', 'metric.test_mcc_fold_2', 'metric.test_precision_fold_5', 'metric.test_precision_fold_2', 'metric.test_auc_fold_7', 'metric.test_auc_fold_6', 'metric.test_recall_fold_7', 'metric.test_auc_fold_5', 'metric.test_auc_fold_0']


## ADDING NEW COLUMNS

In [14]:
# --- Add DATASET Column ---
if 'param.test_counts' in df.columns:
    df['param.test_counts'] = pd.to_numeric(df['param.test_counts'], errors='coerce')
    df['dataset'] = np.where(df['param.test_counts'] < 25, 'DS1', 'DS2')
    print("Added 'dataset' column based on 'param.test_counts'.")
else:
    print("Warning: 'param.test_counts' column not found. Could not create 'dataset' column.")

Added 'dataset' column based on 'param.test_counts'.


In [15]:
# --- Add num_channels Column (MODIFIED LOGIC WITH FALLBACK) ---
# Step 1: Create the fallback values for all rows based on the original inference logic.
print("Inferring fallback 'num_channels' for all rows...")
condition_3c = df['experiment_name'].str.contains("3c", na=False)

if 'param.transfer_learning' in df.columns:
    condition_tl = (df['param.transfer_learning'] == 'True')
else:
    condition_tl = pd.Series([False] * len(df), index=df.index)
    print("Warning: 'param.transfer_learning' column not found for fallback. Assuming False.")

is_3_channels = condition_3c | condition_tl
fallback_channels = pd.Series(np.where(is_3_channels, 3, 4), index=df.index)

# Step 2: Use 'param.num_channels' as the priority source, and fill any missing values with the fallback.
if 'param.num_channels' in df.columns:
    print("Found 'param.num_channels'. Using it as priority, with inference as fallback.")
    # Convert the parameter column to numeric, forcing errors into NaN (Not a Number)
    priority_channels = pd.to_numeric(df['param.num_channels'], errors='coerce')
    # Use the priority values, and for any NaNs, fill with the corresponding value from the fallback Series.
    df['num_channels'] = priority_channels.fillna(fallback_channels)
else:
    print("Did not find 'param.num_channels'. Creating 'num_channels' based on inference only.")
    df['num_channels'] = fallback_channels

# Ensure the final column is an integer type, as channel counts are whole numbers.
df['num_channels'] = df['num_channels'].astype(int)
print("Added 'num_channels' column with priority-fallback logic.")

Inferring fallback 'num_channels' for all rows...
Found 'param.num_channels'. Using it as priority, with inference as fallback.
Added 'num_channels' column with priority-fallback logic.


In [16]:
# --- NEW: Add classi Column ---
# Create a condition to check if the experiment name contains the specific substring
condition_msap = df['experiment_name'].str.contains("_MSA-P_vs_PD_", na=False)

# Use np.where to assign the value based on the condition
df['classes'] = np.where(condition_msap, 'MSA-P vs PD', 'MSA vs PD')
print("Added 'classes' column based on experiment name.")

# --- Correct param.color_transforms based on run_name due to error in logging code---
if 'run_name' in df.columns and 'param.color_transforms' in df.columns:
    # Create a boolean mask for rows where run_name contains the specific string
    color_transform_mask = df['run_name'].str.contains('_color_transforms:True_', na=False)

    # Update the 'param.color_transforms' column to True where the mask is True
    df.loc[color_transform_mask, 'param.color_transforms'] = 'True'
    print("Updated 'param.color_transforms' based on 'run_name' content.")

# --- Create, Display, and Save Final Merged Table ---
# The main 'df' DataFrame now contains all info. We will reorder it for better readability.
# Define the desired column order, including the new 'classes' column
id_cols = ['experiment_name', 'run_name', 'dataset', 'classes', 'num_channels']
# Exclude the original param.num_channels from the list of param columns to avoid redundancy
param_cols = sorted([
    c for c in df.columns
    if c.startswith("param.") and c != 'param.num_channels'
])
metric_cols = sorted([c for c in df.columns if c.startswith("metric.")])

# Construct the final list of columns, ensuring they exist in the DataFrame
final_cols_order = [col for col in id_cols if col in df.columns] + \
                   [col for col in param_cols if col in df.columns] + \
                   [col for col in metric_cols if col in df.columns]

# Create the final, reordered dataframe
results_df = df[final_cols_order].copy()

# Replace 'torchvision' with 'imagenet' in the 'param.pretrained' column
if 'param.pretrained' in results_df.columns:
    results_df['param.pretrained'] = results_df['param.pretrained'].replace('torchvision', 'imagenet')
    print("Replaced 'torchvision' with 'imagenet' in 'param.pretrained' column.")

pd.set_option("display.max_columns", 200)
print("\n--- Final Merged Results Table ---")
# Use display() if in a notebook environment, otherwise print()
try:
    display(results_df)
except NameError:
    print(results_df)

# Save the single, merged DataFrame to a CSV file
results_df.to_csv("mlflow_results_merged.csv", index=False)
print("\nSaved the final merged table to 'mlflow_results_merged.csv'")

print(f"\nLoaded and processed {len(df)} runs from {len(experiments)} experiments at {tracking_uri}")

Added 'classes' column based on experiment name.
Updated 'param.color_transforms' based on 'run_name' content.
Replaced 'torchvision' with 'imagenet' in 'param.pretrained' column.

--- Final Merged Results Table ---


,experiment_name,run_name,dataset,classes,num_channels,param.batch_size,param.best_fold_idx,param.best_lr,param.color_transforms,param.creation_date,param.creation_time,param.crop_percentage,param.dropout_rate,param.epochs,param.fine_tuning,param.freezed_layer_index,param.intensity_augmentation_preset,param.lr_discovery_folds,param.lr_discovery_method,param.mixup_alpha,param.model_library,param.model_name,param.n_folds,param.optimizer_name,param.pretrained,param.test_counts,param.test_pat_ids_for_best_fold,param.test_pat_ids_per_fold,param.total_params,param.train_counts,param.trainable_params,param.transfer_learning,param.use_crop,param.use_lr_discovery,param.val_counts,param.weight_decay,metric.exec_time_min,metric.mean_patient_major_bal_acc,metric.mean_patient_major_mcc,metric.mean_patient_major_precision,metric.mean_patient_major_recall,metric.mean_patient_soft_bal_acc,metric.mean_patient_soft_mcc,metric.mean_patient_soft_precision,metric.mean_patient_soft_recall,metric.mean_test_accuracy,metric.mean_test_auc,metric.mean_test_balanced_acc,metric.mean_test_f1,metric.mean_test_mcc,metric.mean_test_precision,metric.mean_test_recall,metric.patient_major_bal_acc_formatted,metric.patient_major_mcc_formatted,metric.patient_major_precision_formatted,metric.patient_major_recall_formatted,metric.patient_soft_bal_acc_formatted,metric.patient_soft_mcc_formatted,metric.patient_soft_precision_formatted,metric.patient_soft_recall_formatted,metric.std_patient_major_bal_acc,metric.std_patient_major_mcc,metric.std_patient_major_precision,metric.std_patient_major_recall,metric.std_patient_soft_bal_acc,metric.std_patient_soft_mcc,metric.std_patient_soft_precision,metric.std_patient_soft_recall,metric.std_test_accuracy,metric.std_test_auc,metric.std_test_balanced_acc,metric.std_test_f1,metric.std_test_mcc,metric.std_test_precision,metric.std_test_recall
55,DS1_3c_MSA-P_vs_PD_supervised,Densenet121_oversamp_torchvision_10-13_at:22-0...,DS1,MSA-P vs PD,3,32,NaN,0.00012207764786954147,NaN,10-13,22:09:14,0.95,0.2,150,NaN,None,light,4,nested,0,torchvision,Densenet121,8,AdamW,imagenet,14.0,NaN,NaN,6955906,"{0: 92, 1: 93, 2: 91, 3: 91, 4: 91, 5: 90, 6: ...",6955906,False,False,NaN,"{0: 17, 1: 17, 2: 17, 3: 17, 4: 17, 5: 17, 6: ...",1e-4,52.304000,0.656,0.347,0.549,0.750,0.656,0.318,0.562,0.688,0.676000,0.748000,0.663000,0.625000,0.339000,0.565000,0.703000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136,0.288,0.231,0.331,0.205,0.419,0.263,0.325,0.121000,0.173000,0.131000,0.261000,0.273000,0.234000,0.299000
54,DS1_3c_MSA-P_vs_PD_supervised,Densenet169_mixup0.2_oversamp_torchvision_10-1...,DS1,MSA-P vs PD,3,32,NaN,0.00012207764786954147,NaN,10-14,00:00:35,0.95,0.2,150,NaN,None,light,4,nested,0.2,torchvision,Densenet169,8,AdamW,imagenet,14.0,NaN,NaN,12487810,"{0: 92, 1: 93, 2: 91, 3: 91, 4: 91, 5: 90, 6: ...",12487810,False,False,NaN,"{0: 17, 1: 17, 2: 17, 3: 17, 4: 17, 5: 17, 6: ...",1e-4,64.841000,0.700,0.433,0.594,0.844,0.692,0.395,0.629,0.771,0.667000,0.732000,0.672000,0.686000,0.360000,0.615000,0.792000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.198,0.420,0.258,0.329,0.161,0.333,0.169,0.235,0.189000,0.177000,0.185000,0.197000,0.382000,0.173000,0.240000
53,DS1_3c_MSA-P_vs_PD_supervised,Densenet169_oversamp_torchvision_10-14_at:11-5...,DS1,MSA-P vs PD,3,32,NaN,0.00012207764786954147,NaN,10-14,11:55:02,0.95,0.2,150,NaN,None,light,4,nested,0,torchvision,Densenet169,8,AdamW,imagenet,14.0,NaN,NaN,12487810,"{0: 92, 1: 93, 2: 91, 3: 91, 4: 91, 5: 90, 6: ...",12487810,False,False,NaN,"{0: 17, 1: 17, 2: 17, 3: 17, 4: 17, 5: 17, 6: ...",1e-4,57.341000,0.716,0.483,0.625,0.906,0.728,0.487,0.638,0.875,0.682000,0.737000,0.688000,0.708000,0.394000,0.630000,0.823000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.164,0.329,0.166,0.248,0.159,0.330,0.156,0.250,0.163000,0.160000,0.157000,0.162000,0.326000,0.143000,0.196000
52,DS1_3c_MSA-P_vs_PD_supervised,Resnet18_oversamp_torchvision_10-14_at:12-25-18,DS1,MSA-P vs PD,3,32,NaN,0.00012207764786954147,NaN,10-14,12:25:18,0.95,0.2,150,NaN,None,light,4,nes


Saved the final merged table to 'mlflow_results_merged.csv'

Loaded and processed 212 runs from 38 experiments at file:///home/zano/Documents/TESI/FOLDER_CINECA/mlruns


In [ ]:

# df_cleaned = df.drop(columns=columns_to_drop)

Dropping 0 columns containing 'fold': []


In [ ]:

# Adjust if your mlruns is elsewhere
MLRUNS_DIR_CANDIDATES = [
    # "/home/zano/Documents/TESI/FOLDER_CINECA/notebooks/mlruns",
    "/home/zano/Documents/TESI/FOLDER_CINECA/mlruns"
]

# def pick_existing_dir(candidates):
#     for d in candidates:
#         if os.path.isdir(d):
#             return d
#     raise FileNotFoundError(f"No mlruns directory found in: {candidates}")
mlruns_dir = MLRUNS_DIR_CANDIDATES[0] #pick_existing_dir(MLRUNS_DIR_CANDIDATES)
tracking_uri = f"file://{mlruns_dir}"
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient()

experiments = client.search_experiments() 

# --- Configuration for Filtering ---
EXCLUDE_PARAM_KEYS = {"seed", "random_state"}
EXCLUDE_PARAM_PREFIXES = ["debug_", "temp_"]
EXCLUDE_PARAM_REGEX = [r"^unused_.*"]

# --CONFIG FOR METRIC FILTERING
EXCLUDE_METRIC_KEYS = {"train_loss_step", "val_loss_step"}
EXCLUDE_METRIC_PREFIXES = ["raw_", "tmp_"]
EXCLUDE_METRIC_REGEX = [r"val_fold",r".*loss*"]

# --- Data Loading ---
rows = []
for exp in experiments:
    exp_id = exp.experiment_id
    runs = client.search_runs([exp_id], max_results=1000)
    for run in runs:
        info = run.info
        data = run.data
        params = {f"param.{k}": v for k, v in data.params.items()}
        metrics = {f"metric.{k}": v for k, v in data.metrics.items()}
        tags = {f"tag.{k}": v for k, v in data.tags.items()}
        row = {
            "experiment_id": exp_id,
            "experiment_name": exp.name,
            "run_id": info.run_id,
            "run_name": data.tags.get("mlflow.runName", ""),
            "artifact_uri": info.artifact_uri,
        }
        row.update(params)
        row.update(metrics)
        row.update(tags)
        rows.append(row)

# --- DataFrame Creation and Filtering ---
df = pd.DataFrame(rows).sort_values(["experiment_name"])
print(df.shape)
print(df.info())

display(df)

(217, 204)
<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 55 to 203
Columns: 204 entries, experiment_id to metric.test_auc_fold_0
dtypes: float64(153), object(51)
memory usage: 347.5+ KB
None


,experiment_id,experiment_name,run_id,run_name,artifact_uri,param.use_crop,param.test_pat_ids_per_fold,param.crop_percentage,param.dropout_rate,param.creation_time,param.pretrained,param.num_channels,param.transfer_learning,param.intensity_augmentation_preset,param.val_counts,param.trainable_params,param.model_name,param.creation_date,param.best_lr,param.test_pat_ids_for_best_fold,param.batch_size,param.lr_discovery_folds,param.n_folds,param.best_fold_idx,param.weight_decay,param.lr_discovery_method,param.optimizer_name,param.mixup_alpha,param.total_params,param.test_counts,param.train_counts,param.model_library,param.epochs,param.freezed_layer_index,metric.mean_test_accuracy,metric.patient_major_mcc_formatted,metric.patient_major_recall_formatted,metric.mean_patient_soft_precision,metric.std_test_mcc,metric.std_patient_soft_mcc,metric.mean_patient_soft_recall,metric.mean_test_precision,metric.std_patient_soft_bal_acc,metric.mean_test_auc,metric.mean_patient_major_recall,metric.mean_patient_major_precision,metric.std_test_balanced_acc,metric.mean_patient_major_bal_acc,metric.std_test_auc,metric.exec_time_min,metric.std_patient_major_precision,metric.patient_major_precision_formatted,metric.std_patient_major_mcc,metric.mean_test_mcc,metric.std_test_f1,metric.std_test_loss,metric.patient_soft_mcc_formatted,metric.std_test_accuracy,metric.mean_test_balanced_acc,metric.mean_test_recall,metric.std_patient_major_recall,metric.mean_test_f1,metric.patient_major_bal_acc_formatted,metric.mean_patient_soft_bal_acc,metric.std_test_recall,metric.mean_patient_soft_mcc,metric.std_patient_soft_precision,metric.std_patient_major_bal_acc,metric.patient_soft_bal_acc_formatted,metric.std_patient_soft_recall,metric.std_test_precision,metric.patient_soft_precision_formatted,metric.mean_patient_major_mcc,metric.patient_soft_recall_formatted,metric.mean_test_loss,tag.patient_major_mcc_formatted,tag.mlflow.user,tag.mlflow.source.name,tag.patient_major_recall_formatted,tag.mlflow.runName,tag.patient_major_precision_formatted,tag.mlflow.source.type,tag.patient_soft_mcc_formatted,tag.mlflow.log-model.history,tag.patient_major_bal_acc_formatted,tag.mlflow.source.git.commit,tag.patient_soft_bal_acc_formatted,tag.patient_soft_precision_formatted,tag.patient_soft_recall_formatted,metric.val_fold_2/val_f1,metric.val_fold_2/val_mcc,metric.val_fold_2/val_precision,metric.val_fold_2/val_auc,metric.val_fold_2/val_recall,metric.val_fold_2/val_accuracy,metric.val_fold_2/val_balanced_accuracy,metric.val_fold_2/train_accuracy,metric.val_fold_2/train_loss,metric.val_fold_2/val_loss,metric.val_fold_5/val_f1,...,metric.val_fold_5/val_accuracy,metric.val_fold_5/val_balanced_accuracy,metric.val_fold_5/train_accuracy,metric.val_fold_5/train_loss,metric.val_fold_5/val_loss,metric.val_fold_3/val_f1,metric.val_fold_3/val_mcc,metric.val_fold_3/val_precision,metric.val_fold_3/val_auc,metric.val_fold_3/val_recall,metric.val_fold_3/val_accuracy,metric.val_fold_3/val_balanced_accuracy,metric.val_fold_3/train_accuracy,metric.val_fold_3/train_loss,metric.val_fold_3/val_loss,metric.val_fold_7/val_f1,metric.val_fold_7/val_mcc,metric.val_fold_7/val_precision,metric.val_fold_7/val_auc,metric.val_fold_7/val_recall,metric.val_fold_7/val_accuracy,metric.val_fold_7/val_balanced_accuracy,metric.val_fold_7/train_accuracy,metric.val_fold_7/train_loss,metric.val_fold_7/val_loss,metric.val_fold_6/val_f1,metric.val_fold_6/val_mcc,metric.val_fold_6/val_precision,metric.val_fold_6/val_auc,metric.val_fold_6/val_recall,metric.val_fold_6/val_accuracy,metric.val_fold_6/val_balanced_accuracy,metric.val_fold_6/train_accuracy,metric.val_fold_6/train_loss,metric.val_fold_6/val_loss,metric.val_fold_0/val_f1,metric.val_fold_0/val_mcc,metric.val_fold_0/val_precision,metric.val_fold_0/val_auc,metric.val_fold_0/val_recall,metric.val_fold_0/val_accuracy,metric.val_fold_0/val_balanced_accuracy,metric.val_fold_0/train_accuracy,metric.val_fold_0/train_loss,metric.val_fold_0/val_loss,metric.val_fold_1/val_f1,metric.

In [ ]:
import re

# 1. Compile a regex pattern to find 'fold_' + one or more digits
pattern = re.compile(r'fold_\d+')
# 2. Build a list of columns where the pattern is found ANYWHERE
columns_to_drop = [col for col in df.columns if pattern.search(col)]
# 3. Drop the identified columns
df = df.drop(columns=columns_to_drop)

print(f"Dropping {len(columns_to_drop)} columns containing 'fold': {columns_to_drop}")
# 2. Drop the identified columns
# df_cleaned = df.drop(columns=columns_to_drop)

Dropping 0 columns containing 'fold': []
